In [2]:
import subprocess
import os
import time
import pandas as pd
from Bio import SeqIO
from Bio import pairwise2

def compute_alignment_score(seq1, seq2):
    alignments = pairwise2.align.globalxx(seq1.seq, seq2.seq, one_alignment_only=True)
    if alignments:
        alignment = alignments[0]
        return alignment.score
    else:
        return 0  # No alignment found, score is 0

def run_command(command):
    print(f"Running command: {command}")
    start_time = time.time()
    result = subprocess.run(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True)
    elapsed_time = time.time() - start_time
    print(f"Command '{command}' took {elapsed_time:.2f} seconds to run.")
    if result.stdout:
        print(result.stdout)
    if result.stderr:
        print("Error:", result.stderr)
    return result, elapsed_time

def select_best_alignment(sequences, query_sequence):
    best_sequence = None
    best_score = 0

    for sequence in sequences:
        alignment_score = compute_alignment_score(sequence, query_sequence)  # Implement this function to compute alignment score
        if alignment_score > best_score:
            best_score = alignment_score
            best_sequence = sequence

    return best_sequence

def split_fastq(input_path, output_dir, base_name, percentile=20):
    # Read sequences and sort by length
    sequences = list(SeqIO.parse(input_path, "fastq"))
    sequences.sort(key=lambda x: len(x), reverse=True)

    # Split into top percentile and remaining sequences
    split_index = len(sequences) * percentile // 100
    top_sequences = sequences[:split_index]
    remaining_sequences = sequences[split_index:]

    # Write to separate files
    top_sequences_path = os.path.join(output_dir, f"{base_name}_top{percentile}.fastq")
    remaining_sequences_path = os.path.join(output_dir, f"{base_name}_remaining{100-percentile}.fastq")
    SeqIO.write(top_sequences, top_sequences_path, "fastq")
    SeqIO.write(remaining_sequences, remaining_sequences_path, "fastq")

    return top_sequences_path, remaining_sequences_path

def main():
    input_fastq_filename = "rbcL_Qiagen_tomato_5000.fastq"
    input_fastq_path = f"assets/input/{input_fastq_filename}"
    base_name = os.path.splitext(input_fastq_filename)[0]

    output_base_dir = "assets/output"
    output_dir = os.path.join(output_base_dir, base_name)
    os.makedirs(output_dir, exist_ok=True)

    total_time_taken = 0

    # Split the file into top 20% and remaining 80%
    top_sequences_path, remaining_sequences_path = split_fastq(input_fastq_path, output_dir, base_name, percentile=20)

    # Step 1: Align and generate consensus from top 20% sequences
    top_paf_path = os.path.join(output_dir, f"{base_name}_top20_reads.paf")
    top_consensus_path = os.path.join(output_dir, f"{base_name}_top20_consensus.fasta")
    minimap2_command = f"minimap2 -x ava-ont {top_sequences_path} {top_sequences_path} > {top_paf_path}"
    print("Running read alignment with minimap2 on top 20% sequences...")
    _, minimap2_time = run_command(minimap2_command)
    total_time_taken += minimap2_time

    racon_command = f"racon -m 8 -x -6 -g -8 -w 500 {top_sequences_path} {top_paf_path} {top_sequences_path} > {top_consensus_path}"
    print("Generating consensus sequence with racon on top 20% sequences...")
    _, racon_time = run_command(racon_command)
    total_time_taken += racon_time

    # Step 1 bis: If the outputed racon file contains more than one sequence, we take the longest one
    consensus_sequences = list(SeqIO.parse(top_consensus_path, "fasta"))
    if len(consensus_sequences) > 1:
        print(f"Multiple sequences found in {top_consensus_path}. Selecting the best alignment...")
        best_sequence = select_best_alignment(consensus_sequences, consensus_sequences[0])  # Choose the first sequence as the reference
        SeqIO.write(best_sequence, top_consensus_path, "fasta")

    # Step 2: Align the remaining 80% sequences to the top 20% consensus
    remaining_paf_path = os.path.join(output_dir, f"{base_name}_remaining80_reads.paf")
    final_consensus_path = os.path.join(output_dir, f"{base_name}_final_consensus.fasta")
    minimap2_command = f"minimap2 -x map-ont {top_consensus_path} {remaining_sequences_path} > {remaining_paf_path}"
    print("Running read alignment with minimap2 on remaining 80% sequences...")
    _, minimap2_time = run_command(minimap2_command)
    total_time_taken += minimap2_time

    # Step 3: Generate the final consensus sequence with racon
    racon_command = f"racon -m 8 -x -6 -g -8 -w 500 {remaining_sequences_path} {remaining_paf_path} {top_consensus_path} > {final_consensus_path}"
    print("Generating final consensus sequence with racon...")
    _, racon_time = run_command(racon_command)
    total_time_taken += racon_time

    # Print out the total time for each step
    print(f"Minimap2 alignment took {minimap2_time:.2f} seconds.")
    print(f"Total Racon iterations took {total_time_taken - minimap2_time:.2f} seconds.")

    # Print out the total time for the pipeline
    print(f"Total time taken for the pipeline: {total_time_taken:.2f} seconds.")

main()

Running read alignment with minimap2 on top 20% sequences...
Running command: minimap2 -x ava-ont assets/output/rbcL_Qiagen_tomato_5000/rbcL_Qiagen_tomato_5000_top20.fastq assets/output/rbcL_Qiagen_tomato_5000/rbcL_Qiagen_tomato_5000_top20.fastq > assets/output/rbcL_Qiagen_tomato_5000/rbcL_Qiagen_tomato_5000_top20_reads.paf
Command 'minimap2 -x ava-ont assets/output/rbcL_Qiagen_tomato_5000/rbcL_Qiagen_tomato_5000_top20.fastq assets/output/rbcL_Qiagen_tomato_5000/rbcL_Qiagen_tomato_5000_top20.fastq > assets/output/rbcL_Qiagen_tomato_5000/rbcL_Qiagen_tomato_5000_top20_reads.paf' took 2.84 seconds to run.
Error: [M::mm_idx_gen::0.024*1.20] collected minimizers
[M::mm_idx_gen::0.038*1.84] sorted minimizers
[M::main::0.038*1.83] loaded/built the index for 1000 target sequence(s)
[M::mm_mapopt_update::0.040*1.78] mid_occ = 608
[M::mm_idx_stat] kmer size: 15; skip: 5; is_hpc: 0; #seq: 1000
[M::mm_idx_stat::0.041*1.76] distinct minimizers: 61942 (78.57% are singletons); average occurrences: 3.